In [28]:
import psycopg2

# Přihlašovací údaje k databázi
dbname = 'DP_pokus'
user = 'postgres'
password = 'kapli4ky'
host = 'localhost'
port = '5432'  # Výchozí port PostgreSQL

try:
    # Vytvoření připojení
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    # Vytvoření kurzoru pro provádění SQL dotazů
    cursor = connection.cursor()
    print("Připojení k databázi bylo úspěšné")

    # SQL dotaz s opravenou syntaxí a definicí plochy
    sql_query = """
    WITH plochy_veget AS ( 
        SELECT 
            geom,
            ST_Area(geom) AS plocha  
        FROM stav_vegetace
    ),
    plochy_pudy AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_pudy
    ),
    plochy_klima AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_klimatu
    ),
    plochy_mgm AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_intenzity_lidske_cinnosti
    )
    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_veget * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL 
    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Odolnost vegetace vůči suchu' AS podrazene_tema,
        SUM(dry_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Schopnost vegetace bránit erozi' AS podrazene_tema,
        SUM(ero_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Pokryvnost vegetace' AS podrazene_tema,
        SUM(pla_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_soil * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Sklonitost svahu' AS podrazene_tema,
        SUM(w_slo * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Textura' AS podrazene_tema,
        SUM(w_tex * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Skeletovitost' AS podrazene_tema,
        SUM(w_fra * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Hloubka půdy' AS podrazene_tema,
        SUM(w_dep * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Propustnost' AS podrazene_tema,
        SUM(w_dra * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Matečná hornina' AS podrazene_tema,
        SUM(par_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_climate * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Průměrný roční úhrn srážek' AS podrazene_tema,
        SUM(w_rai * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Index sucha' AS podrazene_tema,
        SUM(w_arid * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Orientace svahu' AS podrazene_tema,
        SUM(w_asp * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_mgm * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Intenzita využití půdy' AS podrazene_tema,
        SUM(int * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Hustota populace' AS podrazene_tema,
        SUM(den_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Populační růst' AS podrazene_tema,
        SUM(grw_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom
    """
    cursor.execute(sql_query)
    data = cursor.fetchall()

    # Iterace přes výsledek dotazu a vypsání hodnot
    for row in data:
        if row[1] == 'Celková hodnota':  # Kontrola, zda se jedná o nadřazené téma
            print("Nadřazené téma:", row[0])  # Vypíše nadřazené téma
            print("Hodnota:", row[2])  # Vypíše hodnotu
        else:
            print("Podřazené téma:", row[1])  # Vypíše podřazené téma
            print("Hodnota:", row[2])  # Vypíše hodnotu

except Exception as error:
    print(f"Chyba při připojování k databázi: {error}")

finally:
    if connection:
        cursor.close()
        connection.close()

Připojení k databázi bylo úspěšné
Nadřazené téma: Stav vegetace
Hodnota: 1.5752997003914637
Podřazené téma: Odolnost vegetace vůči suchu
Hodnota: 1.6709904585359474
Podřazené téma: Schopnost vegetace bránit erozi
Hodnota: 1.6007334637586341
Podřazené téma: Pokryvnost vegetace
Hodnota: 1.4930186548935391
Nadřazené téma: Stav půdy
Hodnota: 1.364936518111339
Podřazené téma: Sklonitost svahu
Hodnota: 1.1768975992835449
Podřazené téma: Textura
Hodnota: 1.227805156903358
Podřazené téma: Skeletovitost
Hodnota: 1.5973855249141549
Podřazené téma: Hloubka půdy
Hodnota: 0.9955991567582332
Podřazené téma: Propustnost
Hodnota: 1.8442524906809383
Podřazené téma: Matečná hornina
Hodnota: 1.6163660382022456
Nadřazené téma: Stav klimatu
Hodnota: 1.2582801904727645
Podřazené téma: Průměrný roční úhrn srážek
Hodnota: 1.294279260892862
Podřazené téma: Index sucha
Hodnota: 1.0419274276342088
Podřazené téma: Orientace svahu
Hodnota: 1.510289304529298
Nadřazené téma: Intenzita lidské činnosti
Hodnota: 1.2060

In [30]:
import psycopg2

# Přihlašovací údaje k databázi
dbname = 'DP_pokus'
user = 'postgres'
password = 'kapli4ky'
host = 'localhost'
port = '5432'  # Výchozí port PostgreSQL

# Rozsahy kategorií
rozsahy_kategorii = {
    "N": (1.00, 1.17, 1),
    "P": (1.17, 1.22, 2),
    "F1": (1.22, 1.26, 3),
    "F2": (1.26, 1.32, 4),
    "F3": (1.32, 1.37, 5),
    "C1": (1.37, 1.41, 6),
    "C2": (1.41, 1.53, 7),
    "C3": (1.53, 2.00, 8),
    "N/A": (None, None, 99)  # Pro 0 vody
}

def urci_kategorii(vazeny_prumer):
    if vazeny_prumer is None:
        return f"Kategorie 99 (N/A)"  # Pro 0 vody

    for kod, (minimum, maximum, cislo) in rozsahy_kategorii.items():
        if minimum is not None and maximum is not None:
            if minimum <= vazeny_prumer <= maximum:
                return f"Kategorie {cislo} ({kod})"
        elif kod == "N/A" and vazeny_prumer is None:
            return f"Kategorie 99 (N/A)"

    return "Neznámá kategorie"  # Pro případ, že hodnota nespadá do žádného rozsahu

try:
    # Vytvoření připojení
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    # Vytvoření kurzoru pro provádění SQL dotazů
    cursor = connection.cursor()
    print("Připojení k databázi bylo úspěšné")

    # SQL dotaz s opravenou syntaxí a definicí plochy
    sql_query = """
    WITH plochy_veget AS ( 
        SELECT 
            geom,
            ST_Area(geom) AS plocha 
        FROM stav_vegetace
    ),
    plochy_pudy AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_pudy
    ),
    plochy_klima AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_klimatu
    ),
    plochy_mgm AS (
        SELECT
            geom,
            ST_Area(geom) AS plocha
        FROM stav_intenzity_lidske_cinnosti
    )
    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema, 
        SUM(w_veget * plocha) / SUM(plocha) AS hodnota 
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL 
    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Odolnost vegetace vůči suchu' AS podrazene_tema,
        SUM(dry_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Schopnost vegetace bránit erozi' AS podrazene_tema,
        SUM(ero_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav vegetace' AS nadrazene_tema,
        'Pokryvnost vegetace' AS podrazene_tema,
        SUM(pla_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_vegetace
    JOIN plochy_veget ON stav_vegetace.geom = plochy_veget.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_soil * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Sklonitost svahu' AS podrazene_tema,
        SUM(w_slo * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Textura' AS podrazene_tema,
        SUM(w_tex * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Skeletovitost' AS podrazene_tema,
        SUM(w_fra * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Hloubka půdy' AS podrazene_tema,
        SUM(w_dep * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Propustnost' AS podrazene_tema,
        SUM(w_dra * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav půdy' AS nadrazene_tema,
        'Matečná hornina' AS podrazene_tema,
        SUM(par_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_pudy
    JOIN plochy_pudy ON stav_pudy.geom = plochy_pudy.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_climate * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Průměrný roční úhrn srážek' AS podrazene_tema,
        SUM(w_rai * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Index sucha' AS podrazene_tema,
        SUM(w_arid * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Stav klimatu' AS nadrazene_tema,
        'Orientace svahu' AS podrazene_tema,
        SUM(w_asp * plocha) / SUM(plocha) AS hodnota  
    FROM stav_klimatu
    JOIN plochy_klima ON stav_klimatu.geom = plochy_klima.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Celková hodnota' AS podrazene_tema,  
        SUM(w_mgm * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Intenzita využití půdy' AS podrazene_tema,
        SUM(int * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Hustota populace' AS podrazene_tema,
        SUM(den_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom

    UNION ALL

    SELECT
        'Intenzita lidské činnosti' AS nadrazene_tema,
        'Populační růst' AS podrazene_tema,
        SUM(grw_w * plocha) / SUM(plocha) AS hodnota  
    FROM stav_intenzity_lidske_cinnosti 
    JOIN plochy_mgm ON stav_intenzity_lidske_cinnosti.geom = plochy_mgm.geom
    """
    cursor.execute(sql_query)
    data = cursor.fetchall()

    # Iterace přes výsledek dotazu a vypsání hodnot
    for row in data:
        if row[1] == 'Celková hodnota':  # Kontrola, zda se jedná o nadřazené téma
            print("Nadřazené téma:", row[0])  # Vypíše nadřazené téma
            print("Hodnota:", row[2])  # Vypíše hodnotu
            kategorie = urci_kategorii(row[2])
            print("Kategorie:", kategorie)
        else:
            print("Podřazené téma:", row[1])  # Vypíše podřazené téma
            print("Hodnota:", row[2])  # Vypíše hodnotu
            kategorie = urci_kategorii(row[2])
            print("Kategorie:", kategorie)

except Exception as error:
    print(f"Chyba při připojování k databázi: {error}")

finally:
    if connection:
        cursor.close()
        connection.close()

Připojení k databázi bylo úspěšné
Nadřazené téma: Stav vegetace
Hodnota: 1.5752997003914637
Kategorie: Kategorie 8 (C3)
Podřazené téma: Odolnost vegetace vůči suchu
Hodnota: 1.6709904585359474
Kategorie: Kategorie 8 (C3)
Podřazené téma: Schopnost vegetace bránit erozi
Hodnota: 1.6007334637586341
Kategorie: Kategorie 8 (C3)
Podřazené téma: Pokryvnost vegetace
Hodnota: 1.4930186548935391
Kategorie: Kategorie 7 (C2)
Nadřazené téma: Stav půdy
Hodnota: 1.364936518111339
Kategorie: Kategorie 5 (F3)
Podřazené téma: Sklonitost svahu
Hodnota: 1.1768975992835449
Kategorie: Kategorie 2 (P)
Podřazené téma: Textura
Hodnota: 1.227805156903358
Kategorie: Kategorie 3 (F1)
Podřazené téma: Skeletovitost
Hodnota: 1.5973855249141549
Kategorie: Kategorie 8 (C3)
Podřazené téma: Hloubka půdy
Hodnota: 0.9955991567582332
Kategorie: Neznámá kategorie
Podřazené téma: Propustnost
Hodnota: 1.8442524906809383
Kategorie: Kategorie 8 (C3)
Podřazené téma: Matečná hornina
Hodnota: 1.6163660382022456
Kategorie: Kategori